同样可以通过API来实现softmax回归模型

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

初始化模型参数

这里使用nn.Sequential创建模型，它是一个有序的模块容器，模块将按照定义的顺序被添加到计算图中：
展平层 (nn.Flatten):
    输入的图像是28×28像素的二维图像
    展平层将每个样本从形状为(batch_size, 1, 28, 28)的张量转换为形状为(batch_size, 784)的张量
    这是必要的，因为全连接层需要一维输入

全连接层 (nn.Linear(784, 10)):
    输入特征数为784（展平后的图像像素数）
    输出特征数为10（对应10个类别）
    这个层自动包含权重矩阵和偏置向量，对应于手动实现中的W和b
    数学上，实现了变换：y = Wx + b
    在softmax回归中，这个线性变换的结果将被用来计算类别概率

在这个softmax回归模型中，全连接层既是唯一的计算层，也是输出层

展平层在网络中的位置
    展平层通常位于卷积层/输入层之后，全连接层之前
    它不是真正意义上的"输入层"，而是一个形状转换层
    它没有可学习的参数，只改变数据布局

输入图像 [256, 1, 28, 28]
    ↓
展平层：形状转换
    ↓
展平后 [256, 784]
    ↓
全连接层：线性变换 (W: [784, 10], b: [10])
    ↓
输出 [256, 10]

In [2]:
# PyTorch不会隐式地调整输入的形状。因此，
# 我们在线性层前定义了展平层（flatten），来调整网络输入的形状
net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

在交叉熵损失函数中传递未规范化的预测，并同时计算softmax及其对数， 这是一种类似“LogSumExp技巧”的方式。

能避免上下溢问题

In [3]:
loss = nn.CrossEntropyLoss(reduction='none')

优化算法

In [4]:
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

训练

In [6]:
num_epochs = 10
train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

NameError: name 'Animator' is not defined